# Experimentations
## [INN & CNN with Fashion MNIST]

## Instructions

- Create and checkout a branch for your work (`git checkout -b YOUR_BRANCH_NAME`)
- Create a copy of this template and name it however you want (e.g. `YOUR_NAME.ipynb`).
- Change the title according to what scenario you are testing.
- In the "involution" part, replace the existing model with the relevant model.
- Create as many cells as you deem necessary for the experiments (explain your use case, comparison with convolution or other networks, results, etc.).
- Remove this cell and every "TODO" comment.
- When you are done, commit your changes. Make sure you are only committing changes on the file you created! (you can check with `git status`)
- Push your changes on the repo (`git push --set-upstream origin YOUR_BRANCH_NAME`).
- Create a pull request to the `main` branch so that everyone can read your code before we merge it.

In [ ]:
# cloning the github repo in order to have the file involution.py

!git clone https://github.com/CapucineGARCON/mti-project-involution.git

In [ ]:
%mv /content/mti-project-involution/src/involution.py /content/

In [ ]:
# imports

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from involution import Involution


In [ ]:
from tensorflow.python import keras 
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Dropout, MaxPooling2D, Conv2D

from tensorflow.keras.datasets import fashion_mnist
#import visualkeras

import time

## Datasat importation

We are going to use the Fashion MNIST Dataset which is a famous dataset of Zalando's article images consisting in a training set of 60,000 images and a test set of 10,000 images.
Each observation is a 28 x 28 grayscale image, associated with a label from 10 fashion classes.

In [ ]:
# Import du dataset Fashion MNIST
((X_train, Y_train), (X_test, Y_test)) = fashion_mnist.load_data()

In [ ]:
# Normalization between 0 and 1
X_train, X_test = X_train / 255.0, X_test / 255.0

In [ ]:
# Resizing images (28,28,1)
X_train = np.expand_dims(X_train.astype("float32"), -1) 
X_test = np.expand_dims(X_test.astype("float32"), -1)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
class_names = ['T-shirt/Top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
plt.imshow(X_train[20].reshape(28,28),cmap = 'gray' )
plt.xlabel(class_names[Y_train[20]])
plt.show()

## Convolution

In [ ]:
# Parameters 
epochs = 40
batch_size = 20

In [ ]:
#visualkeras.layered_view(inn, draw_volume=False, legend=True)

In [ ]:

convolution_model = tf.keras.models.Sequential([
          tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', input_shape = (28,28,1), padding = 'same'),
          tf.keras.layers.MaxPool2D((2,2)),
          tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', padding = 'same'),
          tf.keras.layers.MaxPool2D((2,2)),
          tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', padding = 'same'),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(64, activation = 'relu'),
          tf.keras.layers.Dense(10),])


convolution_model.summary()

convolution_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
#def convolution_model():
#  model = Sequential()
#  model.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', input_shape = (28,28,1), padding = 'same'))
#  model.add(MaxPooling2D((2,2)))
#  model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', padding = 'same'))
#  model.add(MaxPooling2D((2,2)))
#  model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', padding = 'same'))
#  model.add(Flatten())
#  model.add(Dense(64, activation = 'relu'))
#  model.add(Dense(10))

In [ ]:
start_time = time.time()

train_convolution = convolution_model.fit(X_train, Y_train, epochs = epochs, validation_data= (X_test, Y_test))
time = time.time() - start_time
print(time)

Training time = 6 minutes

#### Visualization of the loss for the training set and the validation set

In [ ]:
plt.plot(train_convolution.history["loss"], label="Training loss")
plt.plot(train_convolution.history["val_loss"], label="Validation loss")
plt.xlabel('epochs')
plt.ylabel('Loss function')
plt.legend()
plt.show()

We can observe that it is not necessary to train our model on 40 epochs because we are going to have overfitting. Indeed, we can see that our validation loss is incresing after 5 epochs because the model is starting to overfit on the training set.

#### Visualization of the accuraccy for the training and validation set

In [ ]:
plt.plot(train_convolution.history["accuracy"], label="Training accuracy")
plt.plot(train_convolution.history["val_accuracy"], label="Validation accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

## Involution

In [ ]:
involution_model = tf.keras.models.Sequential([
    Involution(channel=3,group_number=1,kernel_size=3,stride=1,reduction_ratio=2),
    tf.keras.layers.ReLU(name="relu1"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    Involution(channel=3,group_number=1,kernel_size=3,stride=1,reduction_ratio=2),
    tf.keras.layers.ReLU(name="relu2"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    Involution(channel=3,group_number=1,kernel_size=3,stride=1,reduction_ratio=2),
    tf.keras.layers.ReLU(name="relu3"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10),
])

involution_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
# Involution Neural Network training
start_time = time.time()

train_involution = involution_model.fit(X_train, Y_train, epochs = epochs, validation_data= (X_test, Y_test))

time = time.time() - start_time
print(time)

The training time is 11 minutes for the involution Neural Network.

In [ ]:
involution_model.summary()

In [ ]:
plt.plot(train_involution.history["loss"], label="Training loss")
plt.plot(train_involution.history["val_loss"], label="Validation loss")
plt.xlabel('Epochs')
plt.ylabel('Loss function')
plt.legend()
plt.show()

In [ ]:
plt.plot(train_involution.history["accuracy"], label="Training accuracy")
plt.plot(train_involution.history["val_accuracy"], label="Validation accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

## Comparison between CNN and INN

In [ ]:
plt.plot(train_involution.history["loss"], label="Inv loss")
plt.plot(train_convolution.history["loss"], label="Conv loss")
plt.xlabel('Epochs')
plt.ylabel('Loss function')
plt.legend()
plt.show()

In [ ]:
plt.plot(train_involution.history["accuracy"], label="Inv accuracy")
plt.plot(train_convolution.history["accuracy"], label="Conv accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()